# NO₂ Station Map for Luxembourg
This notebook loads NO₂ data from local Parquet files and visualizes station locations on a Luxembourg map.

In [ ]:
import pandas as pd
import glob
import os
import folium

In [ ]:
folder_path = 'NO2'
parquet_files = glob.glob(os.path.join(folder_path, '*.parquet'))
df_list = [pd.read_parquet(p) for p in parquet_files]
df = pd.concat(df_list, ignore_index=True)
print('Loaded', len(df), 'rows')

In [ ]:

# Map station identifiers to approximate coordinates
station_coords = {
    'LU0101A': (49.6117, 6.1319),  # Luxembourg City (approx)
    'LU0109A': (49.4950, 5.9800),  # Esch-sur-Alzette (approx)
    'LU0114A': (49.75, 6.1)        # Northern Luxembourg (approx)
}

# Extract station id from Samplingpoint (assumes format 'LU/SPO-LU0101A_...')
df['StationID'] = df['Samplingpoint'].str.extract(r'(LU\d{4}A)')
df['Latitude'] = df['StationID'].map(lambda x: station_coords.get(x, (None, None))[0])
df['Longitude'] = df['StationID'].map(lambda x: station_coords.get(x, (None, None))[1])
print(df[['StationID','Latitude','Longitude']].drop_duplicates())


In [ ]:
m = folium.Map(location=[49.75, 6.1], zoom_start=9)
for _, row in df.dropna(subset=['Latitude','Longitude']).iterrows():
    folium.Marker([row['Latitude'], row['Longitude']], popup=row['StationID']).add_to(m)

m.save('luxembourg_no2_map.html')
m

## Interpolate to raster
Average station readings and interpolate them on a grid covering Luxembourg.

In [ ]:
import numpy as np
from scipy.interpolate import griddata
import rasterio
from rasterio.transform import from_origin

# compute mean value per station
station_means = df.groupby('StationID')['Value'].mean().reset_index()
station_means['Latitude'] = station_means['StationID'].map(lambda x: station_coords.get(x, (None, None))[0])
station_means['Longitude'] = station_means['StationID'].map(lambda x: station_coords.get(x, (None, None))[1])
points = station_means[['Longitude','Latitude']].dropna().values
values = station_means['Value'].dropna().values
lon_min, lon_max = 5.74, 6.53
lat_min, lat_max = 49.44, 50.18
num_pixels = 100
xi = np.linspace(lon_min, lon_max, num_pixels)
yi = np.linspace(lat_min, lat_max, num_pixels)
lon_grid, lat_grid = np.meshgrid(xi, yi)
zi = griddata(points, values, (lon_grid, lat_grid), method='cubic')
transform = from_origin(lon_min, lat_max, (lon_max-lon_min)/num_pixels, (lat_max-lat_min)/num_pixels)
with rasterio.open('no2_luxembourg.tif', 'w', driver='GTiff', height=num_pixels, width=num_pixels, count=1, dtype=zi.dtype, crs='EPSG:4326', transform=transform) as dst:
    dst.write(zi, 1)
print('Saved raster to no2_luxembourg.tif')